'''
v1: Scienceon, DBPIA, NTIS AuthorPapers 접근하여 Rawdata에서 필터링된 papers를 추가
v2: Answer 변경 및 f_id 수정 중
v3: time, tqdm 추가하여 속도 확인
v4: Scienceon, DBPIA에 mng로 검색 추가
v5: DBPIA 삭제하고 KCI 추가 & Rawdata 먼저 접급으로 변경, answer check 추가
'''

In [28]:
from pymongo import MongoClient

client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')
filter_info = client['PUBLIC']['FilterInfo'] #필터접근

f_id = 1 #input
keyid = 632 #keyid

fid_key_query = filter_info.find_one({'fId' : f_id }) #f_id serach

if  f_id > 0 and fid_key_query != None: #f_id check
    for key in fid_key_query.keys() :
        if key == 'nFilter':
            ninst = fid_key_query[key]['inst'] #소속
            nrsc = fid_key_query[key]['rsc'] #공동저자수
            nfund = fid_key_query[key]['fund'] #과제수주비
            nyear = fid_key_query[key]['year'] #연도

        elif key == 'pFilter' :
            pinst = fid_key_query[key]['inst']
            pyear = fid_key_query[key]['year']
            pjournal = fid_key_query[key]['journal']
            plang = fid_key_query[key]['lang']

scion_aut = client['SCIENCEON']['Author']
ntis_aut = client['NTIS']['Author']
kci_aut = client['KCI']['Author']

scion_key_query = client['SCIENCEON']['Rawdata'].find({ 'keyId' : keyid })
ntis_key_query = client['NTIS']['Rawdata'].find({ 'keyId' : keyid })
kci_key_query = client['KCI']['Rawdata'].find({ 'keyId' : keyid })

auts = [scion_aut, ntis_aut, kci_aut] #Author
key_querys = [scion_key_query, ntis_key_query, kci_key_query] #Rawdata
id_domestic = client['ID']['Domestic'] #Domestic

A_id = [] # Author id
all_name = [] # Author name
all_inst = [] # Author inst
Answer_dict = {} # Answer result
site = ['SCIENCEON', 'NTIS', 'KCI']

def case_filter(ft1, ft2, ft3, ft4) :
    if ft1 in ninst or ft1 == "":
        if ft2 < nfund or ft2 == "":
            if ft3 in nyear or ft3 == "":
                if ft4 < nrsc or ft4 == "":
                    return True
    return False
    

for i in range(len(key_querys)):
    mngid_dict = {} #mngid, paper value dict / site마다

    for key_query in key_querys[i]: #rawdata(magid, paper) insert

        if f_id < 1 and fid_key_query == None:
            if key_query['mngId'] not in mngid_dict:
                mngid_dict[key_query['mngId']] = []
            mngid_dict[key_query['mngId']].append(key_query['_id'])

        elif site[i] == 'NTIS' :
            ntis_inst = key_query['ldAgency']
            ntis_rsc = int(key_query['cntRscMan']) + int(key_query['cntRscWom'])
            ntis_fund = key_query['totalFund']
            ntis_year = key_query['prdStart'][:4]
            print(site[i], ntis_inst, ntis_rsc, ntis_fund)
            
            if case_filter(ntis_inst, ntis_fund, ntis_year, ntis_rsc) == True : #필터링
                if key_query['mngId'] not in mngid_dict:
                    mngid_dict[key_query['mngId']] = []
                mngid_dict[key_query['mngId']].append(key_query['_id'])

        else:
            paper_inst = key_query['author_inst'].split(';')[0]
            paper_journal = key_query['journal']
            paper_lang = key_query['issue_lang']
            paper_year =  key_query['issue_year'][:4]
            print(site[i], paper_inst, paper_journal, paper_lang)

            if paper_year in pyear: #필터링
                if key_query['mngId'] not in mngid_dict:
                    mngid_dict[key_query['mngId']] = []
                mngid_dict[key_query['mngId']].append(key_query['_id'])

    paper = []
    aut_querys = auts[i].find({'_id': { '$in' : list(mngid_dict.keys())}})
    for aut_query in aut_querys :
        A_id = aut_query['_id']
        paper = mngid_dict[A_id]
        all_name.append(aut_query['name'])
        all_inst.append(aut_query['inst'].replace("(주) ", "").replace("(주)", "").split(' ')[0])

        Answer = {'fid': f_id, 'keyId': keyid, 'name' : all_name[-1], site[i] : {'inst' : all_inst[-1], 'A_id': A_id, 'papers' : paper} }

        if all_name[-1] not in Answer_dict and all_name[-1]+'_0' not in Answer_dict : #동명이인이 없을 때
            Answer_dict[all_name[-1]] = Answer
        else :
            count = 0
            flag = True
            while flag :
                temp = None 
                tempName = all_name[-1]
              
                if tempName in Answer_dict : # 이름 으로만 key가ㅣ 존재         
                    temp = Answer_dict[tempName]
                    flag = False
                else :
                    tempName = all_name[-1]+'_'+str(count) # 이름 + 숫자로 key가ㅣ 존재
                    if tempName not in Answer_dict :
                        flag = False 
                        break
                    temp = Answer_dict[tempName]
                      
                for key in temp.keys() : # 사이트 돌면서
                    if key != 'name' and key != 'keyId' and key != 'fid' : 
                        src = ""
                        tgt = ""

                        if len(all_inst[-1]) >= len(temp[key]['inst']):
                            src = temp[key]['inst']
                            tgt = all_inst[-1]

                        elif len(all_inst[-1]) < len(temp[key]['inst']):
                            src = all_inst[-1]
                            tgt = temp[key]['inst']

                        if key == site[i] :# 사이트가 동일할때
                            if temp[key]['inst'] == all_inst[-1] or (src != "" and src in tgt) :  # 소속 같을때
                                Answer_dict[tempName][site[i]]['papers'].extend(paper)
                                flag = False
                                break

                            elif all_name[-1]+'_'+str(count+1) not in Answer_dict : #소속이 다를 때
                                Answer_dict[all_name[-1]+'_'+str(count+1)] = Answer

                                if tempName == all_name[-1]:
                                    Answer_dict[all_name[-1]+'_0'] = temp
                                    del Answer_dict[all_name[-1]]

                        else :# 사이트가 다를때 
                            if temp[key]['inst'] == all_inst[-1] or (src != "" and src in tgt):  # 소속 같을때
                                    Answer_dict[tempName][site[i]] =  {'inst' : all_inst[-1], 'A_id': A_id, 'papers' : paper}
                                    flag = False
                                    break
                            
                            elif all_name[-1]+'_'+str(count+1) not in Answer_dict : #소속이 다를 때
                                Answer_dict[all_name[-1]+'_'+str(count+1)] = Answer

                                if tempName == all_name[-1]:
                                    Answer_dict[all_name[-1]+'_0'] = temp
                                    del Answer_dict[all_name[-1]]

                count += 1

#print(sorted(Answer_dict.items()))
print(len(all_name))
#id_domestic.insert_many(Answer_dict.values()) #mongodb 추가

        

SCIENCEON 충주대학교 컴퓨터공학과 한국콘텐츠학회논문지 = The Journal of the Korea Contents Association kor
SCIENCEON 한국과학기술정보연구원 정보기술연구실 정보과학회논문지. Journal of KIISE. 데이타베이스 eng
SCIENCEON 인하대학교 일반대학원 정보공학과 멀티미디어학회논문지 = Journal of Korea Multimedia Society kor
SCIENCEON Korea Institute of Science and Technology Information, Dept. of Knowledge Resources Knowledge Information Center 정보관리학회지 = Journal of the Korean society for information management kor
SCIENCEON Korea Institute of Science and Technology Information, Dept. of Knowledge Resources Knowledge Information Center 정보관리학회지 = Journal of the Korean society for information management kor
SCIENCEON 을지대학교 의료IT마케팅학과 한국인터넷방송통신학회 논문지 = The journal of the Institute of Internet Broadcasting and Communication kor
SCIENCEON 연세대학교 컴퓨터과학과 정보처리학회논문지. The KIPS transactions. Part D. Part D kor
SCIENCEON 인하대학교 정보통신공학과 정보과학회논문지. Journal of KIISE. 컴퓨팅의 실제 및 레터 kor
SCIENCEON 이화여자대학교 일반대학원 문헌정보학과 정보관리학회지 = Journal of the Korean society for information management kor
SCIENCEON

In [9]:
import json

answer = None
numCor = 0
numErr = 0
answerCounter = {}

def ansCheck(result, name) :
    global answer, numCor, numErr, answerCounter
    flag = True

    numK = len(result.keys())    
    if numK-3 != len(answer[name].keys()) :
        flag = False
    else :
        for site in result.keys() :        
            if site != 'name' and site != 'fid' and site != 'keyId':
                if site not in answer[name].keys() :
                    flag = False
                else: 
                    if numK > 4:
                        if answer[name][site] != result[site]['inst'] :
                            flag = False
                    else :
                        if answer[name][site] != result[site]['inst'] and  answer[name][site] != result[site]['inst'].split(' ')[0]:
                            flag = False
    if flag :
        numCor += 1
        answerCounter[name] += 1
        #print(f"Correct \n - Answer : {answer[name]}\n - Result : {result}")
    else :
        print(f"Not Correct \n - Answer : {answer[name]}\n - Result : {result}")
        #if len(answer[name].keys()) > 1 :
        numErr += 1
    return flag

with open('answer632.json', 'r',encoding='utf-8-sig') as a_json :

    answer = json.load(a_json)
    numAns = print(len(answer))
    for key in answer.keys() :
        answerCounter[key] = 0

    for answer_one in Answer_dict:
        test = answer_one
        
        name = Answer_dict[test]['name']
        
        if name in answer :
            ansCheck(Answer_dict[test], name)

        else :
            count = 0
            # if name+str(count) not in answer :
            #     print(name)
            while name+str(count) in answer :
                # if name+count in answer :
                print(f"Check {name+str(count)}")
                if ansCheck(Answer_dict[test], name+str(count)) :
                    break
                count += 1

    print(f"num코렉트 {numCor}")
    print(f"numEror {numErr}")
    tempCount = 0
    '''
    for key in answerCounter.keys() :
        if answerCounter[key] == 0 :
            tempCount += 1
            print(key)
            print(answer[key])
    '''     
    print(tempCount)        
    

156
Not Correct 
 - Answer : {'SCIENCEON': '안양대학교 디지털미디어학과', 'KCI': '안양대학교'}
 - Result : {'fid': 0, 'keyId': 632, 'name': '장석우', 'SCIENCEON': {'inst': '안양대학교', 'A_id': 's100270', 'papers': [ObjectId('61cbf9ba597f83aefb455bcc')]}, 'KCI': {'inst': '안양대학교', 'A_id': 's39040', 'papers': [ObjectId('61cbf9bbf72c82d20d455bc5')]}}
Not Correct 
 - Answer : {'SCIENCEON': '인하공업전문대학교 컴퓨터시스템과'}
 - Result : {'fid': 0, 'keyId': 632, 'name': '이세훈', 'SCIENCEON': {'inst': '인하공업전문대학교', 'A_id': 's102540', 'papers': [ObjectId('61cbf9bf597f83aefb455bcd')]}}
Not Correct 
 - Answer : {'SCIENCEON': '계명대학교 사회과학대학 문헌정보학과', 'KCI': '계명대학교'}
 - Result : {'fid': 0, 'keyId': 632, 'name': '오동근', 'SCIENCEON': {'inst': '계명대학교', 'A_id': 's1351879', 'papers': [ObjectId('61cbf9ce597f83aefb455bdf'), ObjectId('61cbf9ce597f83aefb455be0')]}, 'KCI': {'inst': '계명대학교', 'A_id': 's58372', 'papers': [ObjectId('61cbf9dbf72c82d20d455bff')]}}
Not Correct 
 - Answer : {'SCIENCEON': 'Kunsan National University'}
 - Result : {'fid': 0, 'ke